# Nome: Alberto Rodrigues Ferreira
# Matricula: 391278

In [6]:
import sklearn.datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [7]:
iris_x=sklearn.datasets.load_iris().data
iris_y=sklearn.datasets.load_iris().target
#Padronizando as variaveis.
iris_x=scale(iris_x)

In [8]:
#Orientacao a objeto.
class knn:
    def __init__(self,k):
        self.k=k
        
    def fit(self,xtreino,ytreino):
        self.xtreino=xtreino
        self.ytreino=ytreino
        
    def predict(self,x1):
        previsoes=[]
        #Indices do conjunto de teste
        for i in range(x1.shape[0]):
            distancias=[]
            #indices do conjunto de treinamento
            for j in range(self.xtreino.shape[0]):
                #Calculando a distancia euclidiana da i-esima observacao de teste
                #com todas as observacoes de treino, fiz dois casos pois no caso
                #que somente peco uma observacao de teste as dimensoes mudam. 
                if len(x1)>1:
                    distancias.append(np.sqrt(np.sum((x1[i,:]-self.xtreino[j,:])**2)))
                else:
                   distancias.append(np.sqrt(np.sum((x1-self.xtreino[j,:])**2))) 
            indices_vizinhos=np.argsort(distancias)[:self.k]
            y_candidatos=self.ytreino[indices_vizinhos]
            #A funcao np.unique com return_counts=True dará dois arrays, o 
            #primeiro com os valores unicos e o segundo com a quantidade
            #de ocorrencias de cada um, logo o que aparece mais vezes sera a moda
            #e eu pego o indice da moda e pego a classe que aparece mais vezes.
            indice_da_moda=np.argmax(np.unique(y_candidatos,return_counts=True)[1])
            #A previsao de cada observacao de teste
            previsoes.append(np.unique(y_candidatos)[indice_da_moda])
        return(np.array(previsoes))         
    #Metodo que mede a acuracia do teste e os valores preditos 
    
    def acuracia(self,a,b):
        cont=0
        for i in range(len(b)):
            if a[i]==b[i]:
                cont+=1
        return(np.array([np.round(cont/len(b),3)]))  
        
    #Metodo da matriz de confusao
    def matriz_de_confusao(self,real,predito):
        print(pd.crosstab(real,predito,rownames=["Classe real"]
,colnames=["Classe predita"],margins=True))
    
    #Metodo do knn-edit, elimincacao sequencial
    def edit(self,x_treino,y_treino):
        #Indices do conjunto de treinamento
        indices_treino=np.arange(x_treino.shape[0])
        #Indices que ser das observacoes que vao previstas corretamente e 
        #retiradas.
        indices=[]
        for i in range(x_treino.shape[0]):
            #Chamando a classe knn com k vizinhos para cada observacao de 
            #treinamento.
            modelo=knn(self.k)
            #A funcao np.setdiff1d faz a diferenca entre conjuntos, e np.union1d
            # a uniao, no caso a eliminacao sequencial e feita nessa parte.
            #Primeiro: Para cada observacao de treinamento nunca vai acontecer
            #de os vizinhos conter a mesma observacao.
            #Segundo: Sempre que uma observacao e prevista corretamente ela
            #sera acrescentado na lista indices e assim excluida no resto das
            #iteracoes.
            modelo.fit(x_treino[np.setdiff1d(indices_treino,np.union1d(indices,i)),:],
                               y_treino[np.setdiff1d(indices_treino,np.union1d(indices,i))])
            #Verificando se a observacao e prevista corretamente com o metodo
            #predict da classe.
            if y_treino[i]==modelo.predict(x_treino[[i],:])[0]:
                indices.append(i)
        #Retornando os indices de todo o conjunto de treinamento que sera usado
        #no knn-edit.
        return(np.setdiff1d(indices_treino,indices))

In [9]:
#Fazendo as comparacoes        
#Indices da semente, fazendo 10 particoes aleatorias diferentes.
indices_aleatorios=np.random.randint(1,1000,size=10)

In [10]:
#Listas das acuracias dos dois knn's
acuracia1=[]
acuracia2=[]
#Indices dos k
for i in [2,3,5]:
    for j in indices_aleatorios:
        #Fazendo um modelo da classe knn pros dois e comparando as acuracias.
        modelo=knn(i)   
        xtreino,xteste,ytreino,yteste=train_test_split(iris_x,iris_y,test_size=0.5,random_state=j)     
        modelo.fit(xtreino,ytreino)  
        acuracia1.append(modelo.acuracia(yteste,modelo.predict(xteste))[0])
        
        modelo2=knn(i)
        indices_edit=modelo2.edit(xtreino,ytreino)
        modelo2.fit(xtreino[indices_edit,:],ytreino[indices_edit])
        acuracia2.append(modelo2.acuracia(yteste,modelo2.predict(xteste))[0])


In [11]:
#Tabela com todas as acuracias        
tabela=np.column_stack((acuracia1,acuracia2))

In [12]:
#Separando pelos k
tabela=np.vsplit(tabela,3)

In [13]:
#k=2
#Primeira coluna knn normal, segunda coluna knn-edit.
print("k=2\n",tabela[0])
#Medias das performances
print("Média das perfomances para k=2 :",np.mean(tabela[0],axis=0),"\n")

k=2
 [[0.947 0.853]
 [0.947 0.947]
 [0.893 0.84 ]
 [0.893 0.8  ]
 [0.933 0.84 ]
 [0.947 0.907]
 [0.947 0.933]
 [0.973 0.76 ]
 [0.96  0.813]
 [0.933 0.8  ]]
Média das perfomances para k=2 : [0.9373 0.8493] 



In [14]:
#k=3
print("k=3\n",tabela[1])
print("Média das perfomances para k=3 :",np.mean(tabela[1],axis=0),"\n")

k=3
 [[0.947 0.88 ]
 [0.933 0.907]
 [0.947 0.88 ]
 [0.907 0.893]
 [0.933 0.96 ]
 [0.96  0.773]
 [0.973 0.8  ]
 [0.973 0.88 ]
 [0.973 0.84 ]
 [0.947 0.773]]
Média das perfomances para k=3 : [0.9493 0.8586] 



In [15]:
#k=5
print("k=5\n",tabela[2])
print("Média das perfomances para k=5 :",np.mean(tabela[2],axis=0),"\n")

k=5
 [[0.947 0.987]
 [0.933 0.88 ]
 [0.92  0.893]
 [0.933 0.84 ]
 [0.96  0.8  ]
 [0.96  0.773]
 [0.987 0.667]
 [0.973 0.8  ]
 [0.947 0.747]
 [0.96  0.827]]
Média das perfomances para k=5 : [0.952  0.8214] 

